### Main Page: 

[kaggle](https://www.kaggle.com/c/competitive-data-science-predict-future-sales)

### Purpose

We are asking you to predict total sales for every product and store in the next month. 